In [1]:
%matplotlib inline 

import math
import copy
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import optim, nn

import model
import model_utils
import preprocessing_utils

import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import contextily as ctx
import matplotlib.pyplot as plt

from os.path import dirname, abspath
import os
import sys

#to be put under model.py when I refactor
from tqdm.notebook import tqdm

In [2]:
torch.manual_seed(1234)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Heroku only have cpu

num_timesteps_input = 7 # Default is 30 minutes
num_timesteps_output = 2 # Default is 10 minutes

loss_criterion = nn.MSELoss()

In [4]:
os.getcwd()

'C:\\Users\\daryl\\Desktop\\SUTD school materials\\T7-TheoryAndPracticeOfDeepLearning\\Big Project\\traffic_prediction\\interactive-app'

In [5]:
raw_dir = os.path.join(os.getcwd(), 'data', 'raw')
process_dir = os.path.join(os.getcwd(), 'data', 'processed')

# overwrite = False means that the processing function will only run if the process data files do not exist
# overwrite = True => functions will run regardless
preprocessing_utils.processed(raw_dir, process_dir, overwrite=False)
A, X, metadata, cat2index, timesteps, means, stds = preprocessing_utils.load(process_dir)


test_original_data = X


test_input, test_target = preprocessing_utils.generate_dataset(test_original_data,
                                           num_timesteps_input=num_timesteps_input,
                                           num_timesteps_output=num_timesteps_output)

# input shape (num_samples ,num_vertices, num_timesteps_window, num_features)
# output shape (num_samples ,num_vertices, num_timesteps_window)
adj_mat = preprocessing_utils.get_normalized_adj(A)
adj_mat = torch.from_numpy(adj_mat).to(device)

In [6]:
X.shape

(53, 3, 7)

In [7]:
# indices = [(i, i + (num_timesteps_input + num_timesteps_output)) for i
#                in range(X.shape[2] - (
#                 num_timesteps_input + num_timesteps_output) + 1)]
i = 0
indices = [(i, i + (num_timesteps_input + num_timesteps_output))]

# Save samples
features, target = [], []
for i, j in indices:
    features.append(
        X[:, :, i: i + num_timesteps_input].transpose(
            (0, 2, 1)))
    target.append(X[:, 0, i + num_timesteps_input: j])

In [8]:
test_input = torch.from_numpy(np.array(features))

In [9]:
test_target = torch.from_numpy(np.array(target))

In [10]:
# Load model
saved_models_path = os.path.join(dirname(os.getcwd()), 'saved_models', 'last_saved_model.txt')
with open(saved_models_path) as f:
    saved_model = f.read()

latest_model_path = os.path.join(dirname(os.getcwd()), saved_model)
checkpoint = torch.load(latest_model_path, map_location=None)
model_stgcn = model.Stgcn_Model(checkpoint['model_nodes_num'], checkpoint['model_features_num'], checkpoint['model_input_timesteps'], checkpoint['model_num_output'])
model_stgcn.load_state_dict(checkpoint['state_dict'])
optimizer = optim.Adam(model_stgcn.parameters(), lr=checkpoint['model_lr'])
optimizer = optimizer.load_state_dict(checkpoint['opti_state_dict'])

In [11]:
loaded_model = model_stgcn
loaded_optimizer = optimizer

In [12]:
test_input.shape

torch.Size([1, 53, 7, 3])

In [13]:
test_target.shape

torch.Size([1, 53, 0])

In [14]:
torch.manual_seed(1234)


predicted = model_utils.predict(loaded_model, test_input, adj_mat)
predicted_denorm = preprocessing_utils.denormalize(predicted, stds[0], means[0])

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
predicted

In [ ]:
predicted_denorm

In [ ]:
predicted_denorm = predicted_denorm.numpy()

In [ ]:
predicted_denorm

In [ ]:
predicted_denorm[:,:, :2]

In [ ]:
predicted_denorm.shape

In [ ]:
timestep_speedbands = predicted_denorm.reshape(predicted_denorm.shape[2], predicted_denorm.shape[1])

In [ ]:
timestep_speedbands

In [ ]:
def loc_to_linestring(loc):
    coordArr = loc.split()
    coordArr = [float(coord) for coord in coordArr]
    return LineString([coordArr[1::-1], coordArr[3:1:-1]])

def plotGeoPerformance(metadata, speedbands):
    df = pd.DataFrame(metadata).transpose()
    df["speedbands"] = speedbands
    loc = df["start_pos"] + " " + df["end_pos"]
    linestrings = loc.apply(loc_to_linestring)
    gdf = gpd.GeoDataFrame(df, geometry=linestrings, crs="EPSG:4326")
    gdf = gdf.to_crs('EPSG:3857')
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf.plot(ax=ax, column="speedbands", legend=True, cmap="OrRd", legend_kwds={'label': 'speedbands'})
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ctx.add_basemap(ax)

In [ ]:
plotGeoPerformance(metadata, timestep_speedbands[0])